In [ ]:
from bs4 import BeautifulSoup
import operator
import time
from selenium import webdriver
from collections import defaultdict
import pandas as pd
import numpy as np

from src.styling import *
from src.utils import *
from src.points import *

In [ ]:
def get_stats(leagues, n_weeks):
    last_week_scores = []
    all_weeks_scores = []
    all_scores_dict = defaultdict(list)
    browser = webdriver.Chrome()
    for league in leagues:
        
        espn_scoreboard_url = 'https://fantasy.espn.com/hockey/league/scoreboard'
        urls = [f"{espn_scoreboard_url}?leagueId={league}&matchupPeriodId={i+1}" for i in range(n_weeks)]

        result_lk = defaultdict(list)
        opponent_lk = defaultdict(list)
        result_places = defaultdict(list)
        opponent_places = defaultdict(list)
        for index, url in enumerate(urls):
            browser.get(url)
            time.sleep(7)
            content = browser.page_source
            data = BeautifulSoup(content)
            results = get_week_scores(data)
            league_name = get_league_name(data)
            
            opp_dict = {}
            for sc in results:
                opp_dict[sc[0][0]] = sc[1][0]
                opp_dict[sc[1][0]] = sc[0][0]
                
                all_scores_dict[(sc[0][0], league_name)].append(sc[0][1])
                all_scores_dict[(sc[1][0], league_name)].append(sc[1][1])
            
            scores = get_sorted_week_scores(results)
            
            for score in scores:
                all_weeks_scores.append((score[0], score[1], index+1, league_name))
                if index == n_weeks - 1:
                    last_week_scores.append((score[0], score[1], league_name))
                    
            places = get_places(scores)

            lk = get_luck_score(results, places)
            for item in lk:
                result_places[item].append(places[item] if places[item] % 1.0 != 0 else int(places[item]))
                result_lk[item].append(lk[item] if lk[item] % 1.0 > 1e-7 else int(lk[item]))
            
            for team in opp_dict:
                opp_place = places[opp_dict[team]]
                opponent_places[team].append(opp_place if opp_place % 1.0 != 0 else int(opp_place))
                opp_lk = lk[opp_dict[team]]
                opponent_lk[team].append(opp_lk if opp_lk % 1.0 != 0 else int(opp_lk))

        for key in result_lk:
            result_lk[key].append(np.sum(result_lk[key]))
        for key in result_places:
            result_places[key].append(np.sum(result_places[key]))
        for key in opponent_lk:
            opponent_lk[key].append(np.sum(opponent_lk[key]))
        for key in opponent_places:
            opponent_places[key].append(np.sum(opponent_places[key]))
    
        weeks = [f'Week {i+1}' for i in range(n_weeks)]
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        
        df_lk = pd.DataFrame(data=list(result_lk.values()),
                          index=result_lk.keys(), 
                          columns=weeks + ['SUM'])
        df_lk = df_lk.sort_values(['SUM'])
        display(df_lk.style.set_table_styles(styles).\
                applymap(color_value, subset=weeks).\
                set_caption(f'{league_name}: luck scores'))
        
        df_places = pd.DataFrame(data=list(result_places.values()),
                          index=result_places.keys(), 
                          columns=weeks + ['SUM'])
        df_places = df_places.sort_values(['SUM'])
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        display(df_places.style.set_table_styles(styles).\
                apply(color_place_column, subset=weeks).\
                set_caption(f'{league_name}: places'))                    
        
        df_opp_lk = pd.DataFrame(data=list(opponent_lk.values()),
                                 index=opponent_lk.keys(), 
                                 columns=weeks + ['SUM'])
        df_opp_lk = df_opp_lk.sort_values(['SUM'])
#         display(df_opp_lk.style.set_table_styles(styles).\
#                 applymap(color_opponent_value, subset=weeks).\
#                 set_caption(f'{league_name}: opponent luck scores'))
        
        df_opp_places = pd.DataFrame(data=list(opponent_places.values()),
                          index=opponent_places.keys(), 
                          columns=weeks + ['SUM'])
        df_opp_places = df_opp_places.sort_values(['SUM'])
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
#         display(df_opp_places.style.set_table_styles(styles).\
#                 apply(color_opponent_place_column, subset=weeks).\
#                 set_caption(f'{league_name}: opponent places'))
                                         
    return df_lk, df_places, last_week_scores, all_weeks_scores, all_scores_dict

In [ ]:
leagues = [8290, 31769, 33730, 52338, 57256, 73362809]
a, b, c, d, e = get_stats(leagues, 13)
week_scores_sorted = sorted(c, key=operator.itemgetter(1), reverse=True)

n_top = int(len(week_scores_sorted) / len(leagues))
data = pd.DataFrame(data = week_scores_sorted[:n_top], 
                    index=np.arange(1, 1 + n_top),
                    columns=['Team', 'Score', 'League'])
display(data)

n_show = 20

total_sums = [team + (np.sum(e[team]),) for team in e]
total_sums_sorted = sorted(total_sums, key=operator.itemgetter(2), reverse=True)
data = pd.DataFrame(data = total_sums_sorted[:n_show], 
                    index=np.arange(1, 1 + n_show),
                    columns=['Team', 'Score' ,'League'])
display(data)

all_scores_sorted = sorted(d, key=operator.itemgetter(1), reverse=True)
data = pd.DataFrame(data = all_scores_sorted[:n_show], 
                    index=np.arange(1, 1 + n_show),
                    columns=['Team', 'Score', 'Week' ,'League'])
display(data)

In [ ]:
leagues = [27465869]
a, b, c, d, e = get_stats(leagues, 13)
week_scores_sorted = sorted(c, key=operator.itemgetter(1), reverse=True)

n_top = int(len(week_scores_sorted) / len(leagues))
data = pd.DataFrame(data = week_scores_sorted[:n_top], 
                    index=np.arange(1, 1 + n_top),
                    columns=['Team', 'Score', 'League'])
display(data)
all_scores_soted = sorted(d, key=operator.itemgetter(1), reverse=True)

n_show = 20
data = pd.DataFrame(data = all_scores_soted[:n_show], 
                    index=np.arange(1, 1 + n_show),
                    columns=['Team', 'Score', 'Week' ,'League'])
display(data)